In [1]:
import pandas as pd
import datetime
from os import listdir
import os, os.path
from os.path import isfile, join
import numpy as np
import csv
import re
import pandas as pd
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.log.util import dataframe_utils
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("Log of analysis of time aspects.csv",index_col=0)
df

,case:id,case:concept:name,concept:name,Variant,time:timestamp,Question,Report,Year,Category,Type,Tupla,Organization
0,A1 P2 BPI 2020,A1 P2 BPI 2020,Calculate cycle time,Calculate cycle time of the whole process for ...,2021-11-19 11:20:16.542839,P2,A1,2020,Academic,Analyze Cycle Time,A12020,TechnischeUniversitätBerlin
1,A1 P2 BPI 2020,A1 P2 BPI 2020,Compare cycle time,Compare cycle time,2021-11-19 12:20:16.542839,P2,A1,2020,Academic,Analyze Cycle Time,A12020,TechnischeUniversitätBerlin
2,A2 P2 BPI 2020,A2 P2 BPI 2020,Calculate cycle time,Get cycle time from the previous question,2021-11-19 13:20:16.542839,P2,A2,2020,Student,Analyze Cycle Time,A22020,FacultyofScienceUtrechtUniversity
3,A2 P2 BPI 2020,A2 P2 BPI 2020,Compare cycle time,Compare cycle time,2021-11-19 14:20:16.542839,P2,A2,2020,Student,Analyze Cycle Time,A22020,FacultyofScienceUtrechtUniversity
4,A7 P2 BPI 2020,A7 P2 BPI 2020,Calculate throughput,Calculate throughput,2021-11-19 15:20:16.542839,P2,A7,2020,Academic,Analyze Cycle Time,A72020,GermanResearchCenterforArtificialIntelligenceDFKI
...,...,...,...,...,...,...,...,...,...,...,...,...
943,A19 P1 BPI 2020,A19 P1 BPI 2020,Group traces,Group traces by year,2021-12-28 18:20:16.542839,P1,A19,2020,Professional,Manipulation,A192020,Sberbank
944,A19 P1 BPI 2020,A19 P1 BPI 2020,Group traces,Group traces by activities,2021-12-28 19:20:16.542839,P1,A19,2020,Professional,Manipulation,A192020,Sberbank
945,A19 P1 BPI 2020,A19 P1 BPI 2020,Calculate statistics,Calculate Statistics of activities,2021-12-28 20:20:16.542839,P1,A19,2020,Professional,Calculate statistics,A192020,Sberbank
946,A19 P1 BPI 2020,A19 P1 BPI 2020,Calculate throughput,Calculate throughput,2021-12-28 21:20:16.542839,P1,A19,2020,Professional,Analyze Cycle Time,A192020,Sberbank


In [3]:
def jaccardIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    tamIntersection=len(list(intersection))
    union = set1.union(set2)
    tamUnion=len(list(union))
#     print("similitud: "+str(float(tamIntersection)/tamUnion))
    return float(tamIntersection)/tamUnion

def overlapIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    len1=len(list(set1))
    len2=len(list(set2))
#     print(float(len(list(intersection))))
#     print(min([len1,len2]))
    overlapIndex=float(len(list(intersection)))/min([len1,len2])
    return overlapIndex

def sorensenIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    len1=len(list(set1))
    len2=len(list(set2))
    sorensenIndex=float(2*len(list(intersection)))/(len1+len2)
    return sorensenIndex

In [4]:
df_analisis = df.groupby(by=['case:concept:name'])
# columns=['case:concept:name','Operations','Organization', 'Year', 'Question']
df_agrupado = pd.DataFrame(columns=['case:concept:name','Operations', 'Type', 'Year', 'Question'])
for name, group in df_analisis:
#     org = group.iloc[0]['Category']
    year = group.iloc[0]['Year']
    question = group.iloc[0]['Question']
    lista = group['concept:name'].to_list()
    if(year == 2015):
        tipo = 'Differences'
    elif(year == 2017):
        tipo = 'Waiting time'
    elif(year == 2019):
        tipo='Parts of the process'
    elif(year == 2020):
        if((question == 'P1') or (question == 'P4')):
            tipo = 'Parts of the process'
        elif((question == 'P5') or (question == 'P6')):
            tipo='Bottlenecks'
        elif(question == 'P2'):
            tipo = 'Differences'
    
        
    new_row={'case:concept:name':name, 'Operations':lista, 'Type':tipo, 'Year':year, 'Question':question} 
    df_agrupado = df_agrupado.append(new_row, ignore_index=True)
    
df_agrupado

,case:concept:name,Operations,Type,Year,Question
0,A1 P1 BPI 2017,"[Calculate cycle time, Calculate cycle time, F...",Waiting time,2017,P1
1,A1 P1 BPI 2020,"[Filter traces, Calculate cycle time, Calculat...",Parts of the process,2020,P1
2,A1 P2 BPI 2019,"[Group events by time, Calculate cycle time, R...",Parts of the process,2019,P2
3,A1 P2 BPI 2020,"[Calculate cycle time, Compare cycle time]",Differences,2020,P2
4,A1 P5 BPI 2015,"[Calculate cycle time, Identify organizational...",Differences,2015,P5
...,...,...,...,...,...
121,A8 P1 BPI 2017,"[Filter traces, Calculate waiting time, Repres...",Waiting time,2017,P1
122,A8 P5 BPI 2015,"[Calculate cycle time, Compare cycle time, Gro...",Differences,2015,P5
123,A9 P1 BPI 2017,"[Calculate processing time, Calculate waiting ...",Waiting time,2017,P1
124,A9 P2 BPI 2019,"[Filter traces, Calculate cycle time, Identify...",Parts of the process,2019,P2


In [7]:
df2 = df_agrupado.groupby(by=['Type'])
columns=['analisis', 'type', 'overlap','jaccard','sorensen']
df_metricas_todas = pd.DataFrame(columns=columns)

for name, group in df2:
#     print(name)
    tipo = name
    for i in range(len(group)):   
        for j in range(i+1,len(group)):
            grupo = group.iloc[i]['Operations']
            analisis = group.iloc[i]['case:concept:name'] + "-" + group.iloc[j]['case:concept:name']
#             tipo = 'hola'
            sorensen = sorensenIndex(grupo,group.iloc[j]['Operations'])
            overlap = overlapIndex(grupo,group.iloc[j]['Operations'])
            jaccard = jaccardIndex(grupo,group.iloc[j]['Operations'])
            new_row={'analisis':analisis,'type':tipo ,'overlap':overlap,'jaccard':jaccard,'sorensen':sorensen} 
            df_metricas_todas = df_metricas_todas.append(new_row, ignore_index=True)
        
df_metricas_todas 

,analisis,type,overlap,jaccard,sorensen
0,A1 P5 BPI 2020-A1 P6 BPI 2020,Bottlenecks,0.500000,0.333333,0.500000
1,A1 P5 BPI 2020-A10 P5 BPI 2020,Bottlenecks,1.000000,0.666667,0.800000
2,A1 P5 BPI 2020-A10 P6 BPI 2020,Bottlenecks,0.500000,0.250000,0.400000
3,A1 P5 BPI 2020-A11 P5 BPI 2020,Bottlenecks,1.000000,0.153846,0.266667
4,A1 P5 BPI 2020-A12 P5 BPI 2020,Bottlenecks,1.000000,0.666667,0.800000
...,...,...,...,...,...
2119,A6 P1 BPI 2017-A8 P1 BPI 2017,Waiting time,0.750000,0.230769,0.375000
2120,A6 P1 BPI 2017-A9 P1 BPI 2017,Waiting time,0.571429,0.266667,0.421053
2121,A7 P1 BPI 2017-A8 P1 BPI 2017,Waiting time,0.500000,0.200000,0.333333
2122,A7 P1 BPI 2017-A9 P1 BPI 2017,Waiting time,0.428571,0.250000,0.400000


In [8]:
df_metricas_todas.groupby(['type']).mean()

,overlap,jaccard,sorensen
type,,,
Bottlenecks,0.634015,0.343429,0.469243
Differences,0.431934,0.198521,0.276700
Parts of the process,0.465854,0.188952,0.294283
Waiting time,0.558486,0.274885,0.414164


In [9]:
df2 = df_agrupado
columns=['analisis', 'type', 'overlap','jaccard','sorensen']
df_metricas_todas = pd.DataFrame(columns=columns)

for i in range(len(df2)):
    for j in range(i+1,len(df2)):
        grupo = df2.iloc[i]['Operations']
        analisis = df2.iloc[i]['case:concept:name'] + "-" + df2.iloc[j]['case:concept:name']
#             tipo = 'hola'
        tipo = df2.iloc[i]['Type'] + "-" + df2.iloc[j]['Type']
        sorensen = sorensenIndex(grupo,df2.iloc[j]['Operations'])
        overlap = overlapIndex(grupo,df2.iloc[j]['Operations'])
        jaccard = jaccardIndex(grupo,df2.iloc[j]['Operations'])
        new_row={'analisis':analisis,'type':tipo ,'overlap':overlap,'jaccard':jaccard,'sorensen':sorensen} 
        df_metricas_todas = df_metricas_todas.append(new_row, ignore_index=True)
        
df_metricas_todas 

,analisis,type,overlap,jaccard,sorensen
0,A1 P1 BPI 2017-A1 P1 BPI 2020,Waiting time-Parts of the process,0.333333,0.125000,0.222222
1,A1 P1 BPI 2017-A1 P2 BPI 2019,Waiting time-Parts of the process,0.333333,0.153846,0.266667
2,A1 P1 BPI 2017-A1 P2 BPI 2020,Waiting time-Differences,0.500000,0.142857,0.250000
3,A1 P1 BPI 2017-A1 P5 BPI 2015,Waiting time-Differences,0.333333,0.166667,0.285714
4,A1 P1 BPI 2017-A1 P5 BPI 2020,Waiting time-Bottlenecks,1.000000,0.333333,0.500000
...,...,...,...,...,...
7870,A8 P5 BPI 2015-A9 P2 BPI 2019,Differences-Parts of the process,0.375000,0.214286,0.352941
7871,A8 P5 BPI 2015-A9 P5 BPI 2015,Differences-Differences,0.625000,0.416667,0.588235
7872,A9 P1 BPI 2017-A9 P2 BPI 2019,Waiting time-Parts of the process,0.142857,0.071429,0.133333
7873,A9 P1 BPI 2017-A9 P5 BPI 2015,Waiting time-Differences,0.142857,0.071429,0.133333


In [10]:
df_metricas_todas = df_metricas_todas.replace({'Differences-Bottlenecks': 'Bottlenecks-Differences'})
df_metricas_todas = df_metricas_todas.replace({'Parts of the process-Bottlenecks': 'Bottlenecks-Parts of the process'})
df_metricas_todas = df_metricas_todas.replace({'Waiting time-Bottlenecks':'Bottlenecks-Waiting time'})
df_metricas_todas = df_metricas_todas.replace({'Parts of the process-Differences': 'Differences-Parts of the process'})
df_metricas_todas = df_metricas_todas.replace({'Waiting time-Differences': 'Differences-Waiting time'})
df_metricas_todas = df_metricas_todas.replace({'Waiting time-Parts of the process':'Parts of the process-Waiting time'})

In [11]:
df_metricas_todas.groupby('type').mean()

,overlap,jaccard,sorensen
type,,,
Bottlenecks-Bottlenecks,0.634015,0.343429,0.469243
Bottlenecks-Differences,0.377573,0.158777,0.248568
Bottlenecks-Parts of the process,0.412389,0.173210,0.269281
Bottlenecks-Waiting time,0.586314,0.227788,0.352605
Differences-Differences,0.431934,0.198521,0.276700
Differences-Parts of the process,0.365594,0.146464,0.230079
Differences-Waiting time,0.294624,0.122708,0.197846
Parts of the process-Parts of the process,0.465854,0.188952,0.294283
Parts of the process-Waiting time,0.392760,0.156034,0.251741


In [14]:
df_metricas_todas.groupby('type').var(ddof=0)

C:\Users\maria\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  """Entry point for launching an IPython kernel.


,overlap,jaccard,sorensen
type,,,
Bottlenecks-Bottlenecks,0.092060,0.054110,0.061554
Bottlenecks-Differences,0.103839,0.020385,0.043363
Bottlenecks-Parts of the process,0.098598,0.022975,0.041829
Bottlenecks-Waiting time,0.089382,0.017556,0.029855
Differences-Differences,0.156344,0.058366,0.077269
Differences-Parts of the process,0.104472,0.020918,0.041570
Differences-Waiting time,0.069594,0.016028,0.034541
Parts of the process-Parts of the process,0.100425,0.021335,0.038087
Parts of the process-Waiting time,0.087905,0.014853,0.030437


In [15]:
df_metricas_todas.groupby('type').std(ddof=0)

,overlap,jaccard,sorensen
type,,,
Bottlenecks-Bottlenecks,0.303413,0.232615,0.248101
Bottlenecks-Differences,0.322240,0.142776,0.208237
Bottlenecks-Parts of the process,0.314003,0.151575,0.204522
Bottlenecks-Waiting time,0.298969,0.132498,0.172787
Differences-Differences,0.395403,0.241592,0.277973
Differences-Parts of the process,0.323221,0.144632,0.203887
Differences-Waiting time,0.263806,0.126602,0.185851
Parts of the process-Parts of the process,0.316899,0.146066,0.195159
Parts of the process-Waiting time,0.296488,0.121871,0.174461
